In [63]:
import matplotlib.pyplot as plt
from PIL import Image
import torch.nn as nn
import numpy as np
import os, json, glob

import torch
from torchvision import models, transforms
from torch.autograd import Variable
import torch.nn.functional as F
import sys
from lime import lime_image
from skimage.segmentation import mark_boundaries


In [78]:
# resize and take the center part of image to what our model expects
def get_input_transform():
    normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                    std=[0.229, 0.224, 0.225])       
    transf = transforms.Compose([
        transforms.Resize((256, 256)),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        normalize
    ])    

    return transf

def get_input_tensors(img):
    transf = get_input_transform()
    
    img2 = transf(img).unsqueeze(0)
    # unsqeeze converts single image to batch of 1
    
#     device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#     model.to(device)
#     img = img.to(device)
    
    return img2

def get_pil_transform(): 
    transf = transforms.Compose([
        transforms.Resize((256, 256)),
        transforms.CenterCrop(224)
    ])    

    return transf

def get_preprocess_transform():
    normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                    std=[0.229, 0.224, 0.225])     
    transf = transforms.Compose([
        transforms.ToTensor(),
        normalize
    ])    

    return transf    

def batch_predict(images):
    model.eval()
    batch = torch.stack(tuple(preprocess_transform(i) for i in images), dim=0)

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    batch = batch.to(device)
    
    logits = model(batch)
    probs = F.softmax(logits, dim=1)
    return probs.detach().cpu().numpy()

def get_image(path):
    with open(os.path.abspath(path), 'rb') as f:
        with Image.open(f) as img:
            return img.convert('RGB') 

In [81]:
PATH = r"I:\Research\House\Dataset\resnet84.pth"
feature_extract = True
num_classes = 4

batch_size=32
def set_parameter_requires_grad(model, feature_extracting):
    if feature_extracting:
        for param in model.parameters():
            param.requires_grad = False
            
model_ft = models.resnet101(pretrained=False)
set_parameter_requires_grad(model_ft, feature_extract)
num_ftrs = model_ft.fc.in_features
model_ft.fc = nn.Linear(num_ftrs, num_classes)
input_size = 224
model_ft.load_state_dict(torch.load(PATH))
 
# model_ft.cuda()

model = model_ft 
model.cuda()
model.eval()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [123]:

fi = files[0]

def lime_explain(fi):

    filename = os.path.splitext(os.path.basename(fi))[0]

    print(filename)
        
    img = get_image(fi)
    img_t = get_input_tensors(img)
    
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print("device:", device)
    img_t = img_t.to(device)
#     batch = batch.to(device)
    
    logits = model(img_t)
    _, pred = torch.max(logits, 1)
#     pred = pred.item[0]


    
    print("logits:", logits)
#     print("pred:", pred.cpu().int())
    pred = pred.cpu().item()
    
    print("pred:", pred)
    



    pill_transf = get_pil_transform()
    preprocess_transform = get_preprocess_transform()




    explainer = lime_image.LimeImageExplainer()
    explanation = explainer.explain_instance(np.array(pill_transf(img)), 
                                             batch_predict,
                                             top_labels=5, 
                                             hide_color=0, 
                                             num_samples=1000)


    

    temp, mask = explanation.get_image_and_mask(explanation.top_labels[0], positive_only=True, num_features=5, hide_rest=False)
    img_boundry1 = mark_boundaries(temp/255.0, mask)
    plt.imsave(saved_path + "Pred=" + str(pred) + "_" +filename + '.png', img_boundry1)

    temp, mask = explanation.get_image_and_mask(explanation.top_labels[0], positive_only=False, num_features=10, hide_rest=False)
    img_boundry2 = mark_boundaries(temp/255.0, mask)
    plt.imsave(saved_path + filename + 'redgreen.png', img_boundry2)
    
    del img
    del img_t
    del logits


In [129]:
folder = r'X:\Shared drives\Group_research\Story_research\Datasets\LimeResult\LimeResult\test_img\2\*.jpg'
saved_path = r"X:\Shared drives\Group_research\Story_research\Datasets\LimeResult\LimeResult\test_img\2\results\\"
# saved_path = saved_path.replace(r"\\", '\\')
                                
files = glob.glob(folder)
print(len(files))


for idx, f in enumerate(files[:]):
    print("Processing: ", idx, "/", len(files))
    lime_explain(f)
    

65
Processing:  0 / 65
2129
device: cuda
logits: tensor([[-3.4576, -5.0386, -8.0023, 18.3002]], device='cuda:0',
       grad_fn=<AddmmBackward>)
pred: 3



Processing:  1 / 65
0_Kq4gGNsLehZ2eUkvXyWmdg_-76.583777_39.281887_0_40.52
device: cuda
logits: tensor([[ 2.4643,  0.0746, -5.1131,  3.0677]], device='cuda:0',
       grad_fn=<AddmmBackward>)
pred: 3



Processing:  2 / 65
0_lyMAbzmE1uLNehJ4H_zk6g_-76.617097_39.279067_0_207.24
device: cuda
logits: tensor([[ 2.3298, -3.7812,  2.1722, -0.8420]], device='cuda:0',
       grad_fn=<AddmmBackward>)
pred: 0



Processing:  3 / 65
0_NSy93SUY-AmGISYlBLHDfw_-76.685004_39.284277_0_193.05
device: cuda
logits: tensor([[-0.9942, -1.2368, -0.9195,  3.7783]], device='cuda:0',
       grad_fn=<AddmmBackward>)
pred: 3



Processing:  4 / 65
0_p-WJU0uvSmguyAocsE_J6A_-76.672609_39.313442_0_265.40
device: cuda
logits: tensor([[-1.5613, -4.3093,  1.5574,  4.7607]], device='cuda:0',
       grad_fn=<AddmmBackward>)
pred: 3


...


Processing:  60 / 65
2123
device: cuda
logits: tensor([[-1.7186, -3.1052, -8.8133, 15.3906]], device='cuda:0',
       grad_fn=<AddmmBackward>)
pred: 3



Processing:  61 / 65
2124
device: cuda
logits: tensor([[ 8.4540, -7.0361, -8.3713,  8.1233]], device='cuda:0',
       grad_fn=<AddmmBackward>)
pred: 0



Processing:  62 / 65
2126
device: cuda
logits: tensor([[-0.4384, -5.9283, -7.5944, 15.5726]], device='cuda:0',
       grad_fn=<AddmmBackward>)
pred: 3



Processing:  63 / 65
2127
device: cuda
logits: tensor([[-6.4906, -2.5022, -5.2663, 15.8067]], device='cuda:0',
       grad_fn=<AddmmBackward>)
pred: 3



Processing:  64 / 65
2128
device: cuda
logits: tensor([[-1.5300, -5.2207, -5.1962, 12.8762]], device='cuda:0',
       grad_fn=<AddmmBackward>)
pred: 3
